In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import trimesh

scene = trimesh.Scene()

In [ ]:
import my_code.diffusion_training_sign_corr.data_loading as data_loading

train_dataset_template = data_loading.get_val_dataset(
    'FAUST_orig', 'train', 128, preload=False, return_evecs=True, centering='bbox'
    )[1]
test_dataset_single = data_loading.get_val_dataset(
    'FAUST_orig', 'test', 128, preload=False, return_evecs=True, centering='bbox'
)[0]
test_dataset_template = data_loading.get_val_dataset(
    'FAUST_orig', 'test', 128, preload=False, return_evecs=True, centering='bbox'
)[1]

test_dataset_single = [test_dataset_single[i] for i in range(len(test_dataset_single))]

In [ ]:
import networks.diffusion_network as diffusion_network
import yaml


# exp_name = 'signNet_64_remeshed_mass_6b_1-2ev_10_0.2_0.8'
# exp_name = 'signNet_64_FAUST_orig_1k'
exp_name = 'signNet_32_FAUST_orig'

# exp_name = 'signNet_remeshed_mass_6b_1ev_10_0.2_0.8'

exp_dir = f'/home/s94zalek_hpc/shape_matching/my_code/experiments/sign_net/{exp_name}'

with open(f'{exp_dir}/config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
start_dim = config['start_dim']
feature_dim = config['feature_dim']
evecs_per_support = config['evecs_per_support']


device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = diffusion_network.DiffusionNet(
    **config['net_params']
    ).to(device)

# Test the accuracy

In [ ]:
import my_code.sign_canonicalization.test_sign_correction as test_sign_correction

err_per_iter = []
# # for n_iter in range(0, 4000 + 1, 400):
# for n_iter in [0, 100, 200, 300, 400, 500, 600, 700, 800, 900]:
for n_iter in [0, 4000]:

    net.load_state_dict(torch.load(f'{exp_dir}/{n_iter}.pth'))
    
    mean_incorrect_signs, _ = test_sign_correction.test_on_dataset(
        net, test_dataset_single,
        config=config, n_epochs=100)
    
    print(f'{n_iter}.pth: {mean_incorrect_signs:.2f}')
    err_per_iter.append((n_iter, mean_incorrect_signs, _))


In [ ]:
for (n_iter, mean_incorrect_signs, _) in err_per_iter:
    print(f'{n_iter}.pth: {mean_incorrect_signs / 32 * 100:.2f}%')

# Plot the summary vectors

In [ ]:
import my_code.sign_canonicalization.training as sign_training

# net.load_state_dict(torch.load(f'{exp_dir}/2000.pth'))
net.load_state_dict(torch.load(f'{exp_dir}/4000.pth'))
# net.load_state_dict(torch.load(f'{exp_dir}/50000.pth'))

# for curr_idx in range(len(test_dataset_single)):
for curr_idx in [11]:

    ##############################################
    # Select a shape
    ##############################################

    train_shape = test_dataset_single[curr_idx]           
    
    ##############################################
    # Set the variables
    ##############################################

    # train_shape = double_shape['second']
    verts = train_shape['verts'].unsqueeze(0).to(device)
    faces = train_shape['faces'].unsqueeze(0).to(device)    

    evecs_orig = train_shape['evecs'].unsqueeze(0)[:, :, config['start_dim']:config['start_dim']+config['feature_dim']].to(device)
    
    if 'with_mass' in config and config['with_mass']:
        mass_mat = torch.diag_embed(
            train_shape['mass'].unsqueeze(0)
            ).to(device)
    else:
        mass_mat = None

    # predict the sign change
    with torch.no_grad():
        sign_pred_0, supp_vec_0, prod_0 = sign_training.predict_sign_change(
            net, verts, faces, evecs_orig, 
            mass_mat=mass_mat, input_type=net.input_type,
            evecs_per_support=config['evecs_per_support'],
            mass=train_shape['mass'].unsqueeze(0), L=train_shape['L'].unsqueeze(0),
            evals=train_shape['evals'].unsqueeze(0), evecs=train_shape['evecs'].unsqueeze(0),
            gradX=train_shape['gradX'].unsqueeze(0), gradY=train_shape['gradY'].unsqueeze(0)
            )
        
    if 'with_mass' in config and config["with_mass"]:

        print('Using mass')

        supp_vec_norm = torch.nn.functional.normalize(
            supp_vec_0[0].transpose(0, 1) \
                @ mass_mat[0],
            p=2, dim=1)
        
        evecs_cond = supp_vec_norm @ evecs_orig[0]
        supp_vec_norm = supp_vec_norm.transpose(0, 1).unsqueeze(0)

    else:
        print('Not using mass')
        
        supp_vec_norm = torch.nn.functional.normalize(
            supp_vec_0[0].transpose(0, 1),
            p=2, dim=1)
        evecs_cond = supp_vec_norm @ evecs_orig[0]
        supp_vec_norm = supp_vec_norm.transpose(0, 1).unsqueeze(0)

        
        
    break

In [ ]:
scene.geometry.clear()

# material=trimesh.visual.material.SimpleMaterial(
#         image=None,
#         diffuse=[240] * 4,
#         # glossiness=0.5,
#     )
# mesh1.visual.material = material


evec_id = 15

# supp_vec = supp_vec_0[0, :, evec_id].cpu()
supp_vec = supp_vec_norm[0, :, evec_id].cpu()

# supp_vec is a vector in [-1, 1]
# make that the minimum negative value and maximum positive value have the same absolute value
# but the zero value is still zero
max_abs = torch.max(torch.abs(supp_vec))

idx_min = torch.argmin(supp_vec)
idx_max = torch.argmax(supp_vec)

supp_vec[idx_min] = -max_abs
supp_vec[idx_max] = max_abs


mesh1 = trimesh.Trimesh(verts[0].cpu().numpy(), faces[0].cpu().numpy())
cmap1 = trimesh.visual.color.interpolate(supp_vec, 'bwr')

# smooth the colors
# cmap1 = (cmap1.astype(np.int32) + np.roll(cmap1.astype(np.int32), 1) + np.roll(cmap1.astype(np.int32), -1)) / 3
# cmap1 = cmap1.clip(0, 255).astype(np.uint8)

cmap1_faces = trimesh.visual.color.vertex_to_face_color(cmap1, mesh1.faces)
mesh1.visual.face_colors = cmap1_faces.clip(0, 255).astype(np.uint8)
# mesh1.visual.vertex_colors = cmap1[:len(mesh1.vertices)].clip(0, 255).astype(np.uint8)

mesh2 = trimesh.Trimesh(verts[0].cpu().numpy() + np.array([1, 0, 0]), faces[0].cpu().numpy())
cmap2 = trimesh.visual.color.interpolate(evecs_orig[0, :, evec_id].cpu().numpy(), 'bwr')
# mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)].clip(0, 255).astype(np.uint8)

cmap2_faces = trimesh.visual.color.vertex_to_face_color(cmap2, mesh2.faces)
mesh2.visual.face_colors = cmap2_faces.clip(0, 255).astype(np.uint8)

scene.add_geometry(mesh1)
scene.add_geometry(mesh2)

scene.set_camera(angles=(-0.5, 0, 0), distance=(1.7), center=(0.5, 0, 0), resolution=None, fov=None)

light = trimesh.scene.lighting.DirectionalLight(color=[255, 255, 255, 255])
scene.lights = [light]

print(f'evec n {evec_id + 1}')
print(f'projection {prod_0[0][evec_id, evec_id].item():.2f}')

scene.show()

In [ ]:
import my_code.utils.plotting_utils as plotting_utils

l = 0
h = 64

fig, axs = plt.subplots(1, 1, figsize=(8, 8))

plotting_utils.plot_Cxy(fig, axs, prod_0[0].cpu(), 'Projection', l, h, show_grid=False, show_colorbar=False)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].plot(supp_vec_norm[0, :, 16].cpu().numpy())
ax[1].plot(supp_vec)

# plt.plot(supp_vec_0[0, :, 16].cpu())
plt.show()

In [ ]:
from tqdm import tqdm

net.load_state_dict(torch.load(f'{exp_dir}/4000.pth'))

tqdm._instances.clear()

C_xy_pred_list = torch.tensor([])
C_xy_orig_list = torch.tensor([])
prod_with_support_list = torch.tensor([])
supp_vec_list = torch.tensor([])

data_0 = train_dataset_template[0]
verts_first = data_0['first']['verts'].unsqueeze(0).to(device)
faces_first = data_0['first']['faces'].unsqueeze(0).to(device)
evecs_first = data_0['first']['evecs'][:, start_dim:start_dim+feature_dim].unsqueeze(0).to(device)
corr_first = data_0['first']['corr']
mass_mat_first = torch.diag_embed(data_0['first']['mass'].unsqueeze(0)).to(device)


with torch.no_grad():
    sign_pred_first = sign_training.predict_sign_change(
        net, verts_first, faces_first, evecs_first, 
        mass_mat=mass_mat_first, input_type=net.input_type,
        evecs_per_support=config['evecs_per_support'],
        mass=data_0['first']['mass'].unsqueeze(0), L=data_0['first']['L'].unsqueeze(0),
        evals=data_0['first']['evals'].unsqueeze(0), evecs=data_0['first']['evecs'].unsqueeze(0),
        gradX=data_0['first']['gradX'].unsqueeze(0), gradY=data_0['first']['gradY'].unsqueeze(0)
        )[0]


for i in tqdm(range(len(train_dataset_template))):
    # data_0 = test_dataset[12]
    data_0 = train_dataset_template[i]

    verts_second = data_0['second']['verts'].unsqueeze(0).to(device)
    faces_second = data_0['second']['faces'].unsqueeze(0).to(device)
    evecs_second = data_0['second']['evecs'][:, start_dim:start_dim+feature_dim].unsqueeze(0).to(device)
    corr_second = data_0['second']['corr']
    
    mass_mat_second = torch.diag_embed(data_0['second']['mass'].unsqueeze(0)).to(device)

    C_gt_xy = data_0['second']['C_gt_xy'][0]

    # predict the sign change
    with torch.no_grad():
        sign_pred_second, supp_vec_second, prod_with_support = sign_training.predict_sign_change(
            net, verts_second, faces_second, evecs_second, 
            mass_mat=mass_mat_second, input_type=net.input_type,
            evecs_per_support=config['evecs_per_support'],
            mass=data_0['second']['mass'].unsqueeze(0), L=data_0['second']['L'].unsqueeze(0),
            evals=data_0['second']['evals'].unsqueeze(0), evecs=data_0['second']['evecs'].unsqueeze(0),
            gradX=data_0['second']['gradX'].unsqueeze(0), gradY=data_0['second']['gradY'].unsqueeze(0)
            )

    # functional maps
    C_xy_pred = torch.linalg.lstsq(
        evecs_second.cpu()[0, corr_second] * torch.sign(sign_pred_second).cpu(),
        evecs_first.cpu()[0, corr_first] * torch.sign(sign_pred_first).cpu()
        ).solution
    
    C_xy_pred_list = torch.cat([C_xy_pred_list, C_xy_pred.unsqueeze(0)])
    C_xy_orig_list = torch.cat([C_xy_orig_list, C_gt_xy.unsqueeze(0)])
    
    # conditioning
    
    evecs_second_corrected = evecs_second.cpu()[0] * torch.sign(sign_pred_second).cpu()
    evecs_second_corrected_norm = evecs_second_corrected / torch.norm(evecs_second_corrected, dim=0, keepdim=True)
    
    evecs_cond_second = torch.nn.functional.normalize(
        supp_vec_second[0].cpu().transpose(0, 1) \
            @ mass_mat_second[0].cpu(),
        p=2, dim=1) \
            @ evecs_second_corrected_norm 
    
    prod_with_support_list = torch.cat([prod_with_support_list, evecs_cond_second.cpu().unsqueeze(0)])
    
    
    
    
    
    # prod_with_support_list = torch.cat([prod_with_support_list, prod_with_support.cpu()])


for i in tqdm(range(len(test_dataset_template))):
    # data_0 = test_dataset[12]
    data_0 = test_dataset_template[i]

    verts_second = data_0['second']['verts'].unsqueeze(0).to(device)
    faces_second = data_0['second']['faces'].unsqueeze(0).to(device)
    evecs_second = data_0['second']['evecs'][:, start_dim:start_dim+feature_dim].unsqueeze(0).to(device)
    corr_second = data_0['second']['corr']
    
    mass_mat_second = torch.diag_embed(data_0['second']['mass'].unsqueeze(0)).to(device)

    C_gt_xy = data_0['second']['C_gt_xy'][0]

    # predict the sign change
    with torch.no_grad():
        sign_pred_second, supp_vec_second, prod_with_support = sign_training.predict_sign_change(
            net, verts_second, faces_second, evecs_second, 
            mass_mat=mass_mat_second, input_type=net.input_type,
            evecs_per_support=config['evecs_per_support'],
            mass=data_0['second']['mass'].unsqueeze(0), L=data_0['second']['L'].unsqueeze(0),
            evals=data_0['second']['evals'].unsqueeze(0), evecs=data_0['second']['evecs'].unsqueeze(0),
            gradX=data_0['second']['gradX'].unsqueeze(0), gradY=data_0['second']['gradY'].unsqueeze(0)
            )

    C_xy_pred = torch.linalg.lstsq(
        evecs_second.cpu()[0, corr_second] * torch.sign(sign_pred_second).cpu(),
        evecs_first.cpu()[0, corr_first] * torch.sign(sign_pred_first).cpu()
        ).solution
    
    C_xy_pred_list = torch.cat([C_xy_pred_list, C_xy_pred.unsqueeze(0)])
    C_xy_orig_list = torch.cat([C_xy_orig_list, C_gt_xy.unsqueeze(0)])
    # prod_with_support_list = torch.cat([prod_with_support_list, prod_with_support.cpu()])


    # conditioning
    
    evecs_second_corrected = evecs_second.cpu()[0] * torch.sign(sign_pred_second).cpu()
    evecs_second_corrected_norm = evecs_second_corrected / torch.norm(evecs_second_corrected, dim=0, keepdim=True)
    
    evecs_cond_second = torch.nn.functional.normalize(
        supp_vec_second[0].cpu().transpose(0, 1) \
            @ mass_mat_second[0].cpu(),
        p=2, dim=1) \
            @ evecs_second_corrected_norm 
    
    prod_with_support_list = torch.cat([prod_with_support_list, evecs_cond_second.cpu().unsqueeze(0)])
    


In [ ]:
prod_with_support_list.shape

In [ ]:
import my_code.utils.plotting_utils as plotting_utils

l = 0
h = 32

fig, axs = plt.subplots(1, 3, figsize=(15, 5))

plotting_utils.plot_Cxy(fig, axs[0], C_xy_orig_list[81], 'C_xy_orig', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[1], C_xy_pred_list[81], 'C_xy_pred', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[2], prod_with_support_list[81], 'prod_with_support', l, h, show_grid=False, show_colorbar=False)


# plotting_utils.plot_Cxy(fig, axs[1, 0], C_xy_orig_list[82], 'C_xy_pred', l, h, show_grid=False, show_colorbar=False)
# plotting_utils.plot_Cxy(fig, axs[1, 1], C_xy_pred_list[82], 'C_xy_pred', l, h, show_grid=False, show_colorbar=False)


plt.show()

In [ ]:
import importlib

importlib.reload(plotting_utils)

In [ ]:
import my_code.utils.plotting_utils as plotting_utils
import os

l = 0
h = 32

fig, axs = plt.subplots(1, figsize=(5, 5))

# make one figure and axes
# fig, axs = plt.subplots(1, 5, figsize=(15, 4))

# plotting_utils.plot_Cxy(fig, axs[0], C_xy_orig_list[81], 'C_xy_orig', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs, C_xy_pred_list[73], title=None, min_dim=l, max_dim=h, show_grid=False, show_colorbar=False)
# plotting_utils.plot_Cxy(fig, axs[2], prod_with_support_list[81], 'prod_with_support', l, h, show_grid=False, show_colorbar=False)


# plotting_utils.plot_Cxy(fig, axs[1, 0], C_xy_orig_list[82], 'C_xy_pred', l, h, show_grid=False, show_colorbar=False)
# plotting_utils.plot_Cxy(fig, axs[1, 1], C_xy_pred_list[82], 'C_xy_pred', l, h, show_grid=False, show_colorbar=False)

# get 5 random indices without replacement
# rand_idx = np.random.choice(len(C_xy_orig_list), 5, replace=False)

# print(rand_idx)

# for i in range(5):
#     plotting_utils.plot_Cxy(fig, axs[i], C_xy_pred_list[rand_idx[i]], title=None, min_dim=l, max_dim=h, show_grid=False, show_colorbar=False)

# remove axis ticks
axs.set_xticks([])
axs.set_yticks([])

base_dir = f'/home/s94zalek_hpc/shape_matching/notebooks/rebuttal/noisy_fmaps'
os.makedirs(base_dir, exist_ok=True)
# get number of files in the directory
# files = os.listdir(base_dir)

# plt.show()

plt.savefig(f'{base_dir}/Cxy_73.png', bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1, figsize=(5, 5))

random_noise = torch.randn_like(C_xy_pred_list[73]) * 0.15

plotting_utils.plot_Cxy(fig, axs, random_noise, title=None, min_dim=l, max_dim=h, show_grid=False, show_colorbar=False)

# remove axis ticks
axs.set_xticks([])
axs.set_yticks([])

base_dir = f'/home/s94zalek_hpc/shape_matching/my_code/figures'
# get number of files in the directory
files = os.listdir(base_dir)

# plt.show()

plt.savefig(f'{base_dir}/rand_{len(files)}.png', bbox_inches='tight')

# plt.show()

In [ ]:
fig, axs = plt.subplots(1, figsize=(5,5))

random_noise = torch.randn_like(C_xy_pred_list[73]) * 0.1


for i in range(10):
    noisy_fmap = C_xy_pred_list[73] * (i + 3) / 10 + random_noise * (13 - i) / 10

    # normalize between -1 and 1
    # noisy_fmap = noisy_fmap / torch.max(torch.abs(noisy_fmap))

    plotting_utils.plot_Cxy(fig, axs, noisy_fmap, title=None, min_dim=l, max_dim=h, show_grid=False, show_colorbar=False)

    # remove axis ticks
    axs.set_xticks([])
    axs.set_yticks([])

    # plt.show()

    plt.savefig(f'{base_dir}/Cxy_rand_{i}.png', bbox_inches='tight')

# plt.show()

In [ ]:
# apply standard scaling and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd

import matplotlib as mpl

# plt.rcParams['text.usetex'] = True

mpl.rcParams.update(mpl.rcParamsDefault)

# set the font to be Times New Roman
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

scaler = StandardScaler()
pca = PCA(n_components=4)

C_xy_pred_list_scaled = scaler.fit_transform(C_xy_pred_list.reshape(C_xy_pred_list.shape[0], -1))
C_xy_pred_list_pca = pca.fit_transform(C_xy_pred_list_scaled)
pca_df_pred = pd.DataFrame(C_xy_pred_list_pca[:, :2], columns=[f'PC {i}' for i in range(2)])
pca_df_pred['class'] = [i // 10 for i in range(C_xy_pred_list_pca.shape[0])]


C_xy_orig_list_scaled = scaler.fit_transform(C_xy_orig_list.reshape(C_xy_orig_list.shape[0], -1))
C_xy_orig_list_pca = pca.fit_transform(C_xy_orig_list_scaled)
pca_df_orig = pd.DataFrame(C_xy_orig_list_pca[:, :2], columns=[f'PC {i}' for i in range(2)])
pca_df_orig['class'] = [i // 10 for i in range(C_xy_orig_list_pca.shape[0])]

prod_with_support_list_scaled = scaler.fit_transform(prod_with_support_list.reshape(prod_with_support_list.shape[0], -1))
prod_with_support_list_pca = pca.fit_transform(prod_with_support_list_scaled)
pca_df_prod = pd.DataFrame(prod_with_support_list_pca[:, :2], columns=[f'PC {i}' for i in range(2)])
pca_df_prod['class'] = [i // 10 for i in range(prod_with_support_list_pca.shape[0])]


fig, axs = plt.subplots(1, 3, figsize=(18, 5))


sns_plot_orig = sns.scatterplot(data=pca_df_orig, x='PC 0', y='PC 1', hue='class', palette='tab10',
                s=80, ax=axs[0],
                legend=False
)
sns_plot_pred = sns.scatterplot(data=pca_df_pred, x='PC 0', y='PC 1', hue='class', palette='tab10',
                s=80, ax=axs[1],
                # legend=False
)
sns_plot_prod = sns.scatterplot(data=pca_df_prod, x='PC 0', y='PC 1', hue='class', palette='tab10',
                s=80, ax=axs[2],
                legend=False
                # legend should be horizontal
                # legend='brief'
)

# put title under the plot instead of above
axs[0].set_title('$C_{1T}$ before Sign Correction', y=-0.1, fontsize=16)
axs[1].set_title('$C_{1T}$ after Sign Correction', y=-0.1, fontsize=16)
axs[2].set_title('Conditioning $y$', y=-0.1, fontsize=16)


plt.setp(axs[1].get_legend().get_texts(), fontsize='16')
plt.setp(axs[1].get_legend().get_title(), fontsize='16')

# axs[0].title.set_position([.5, 1.05])

axs[0].set_xlim(-20, 20)
axs[0].set_ylim(-20, 20)

# axs[1].set_xlim(-20, 25)
# axs[1].set_ylim(-20, 25)

# axs[2].set_xlim(-20, 30)
# axs[2].set_ylim(-25, 25)

# remove axis labels
for ax in axs:
    ax.set_xlabel('')
    ax.set_ylabel('')

# set all x and y ticks to be multiples of 10
for ax in axs:
    min_x, max_x = ax.get_xlim()
    min_y, max_y = ax.get_ylim()
    
    # ax.set_xticks(np.arange(np.ceil(min_x / 10) * 10, np.floor(max_x / 10) * 10 + 1, 10))
    # ax.set_yticks(np.arange(np.ceil(min_y / 10) * 10, np.floor(max_y / 10) * 10 + 1, 10))
    
    ax.set_xticks([])
    ax.set_yticks([])
    

sns.move_legend(sns_plot_pred, 'upper right', bbox_to_anchor=(2.6, 1), ncol=1, fontsize=14)
# sns.move_legend(sns_plot_prod, 'lower center', bbox_to_anchor=(0, -0.2), ncol=len(pca_df_pred['class'].unique()))


# make legend horizontal
# handles, labels = axs[1].get_legend_handles_labels()
# axs[1].legend(handles=handles[1:], labels=labels[1:], loc='upper right', bbox_to_anchor=(1.5, 1), ncol=1)



print('PCA on predicted')
plt.show()



### Other code

In [ ]:
# apply standard scaling and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd


scaler = StandardScaler()
pca = PCA(n_components=32)

C_xy_orig_list_scaled = scaler.fit_transform(C_xy_orig_list.reshape(C_xy_orig_list.shape[0], -1))
C_xy_orig_list_pca = pca.fit_transform(C_xy_orig_list_scaled)


fig, axs = plt.subplots(1, 1, figsize=(5, 5))

# plot explained variance
axs.plot(pca.explained_variance_ratio_, '.-')
axs.set_title('Explained variance ratio')


pca_df = pd.DataFrame(C_xy_orig_list_pca[:, :2], columns=[f'PCA_{i}' for i in range(2)])

# remove the outliers based on standard deviation
# pca_df = pca_df[(np.abs(pca_df) < 50).all(axis=1)]


# pca_df['name'] = names_y
pca_df['body_type'] = [i // 10 for i in range(C_xy_orig_list_pca.shape[0])]
# pca_df['pose'] = [i % 10 for i in range(C_xy_orig_list_pca.shape[0])]

pca_df = pca_df[(np.abs(pca_df) < np.std(pca_df) * 6
                 ).all(axis=1)]

# use numbers as markers
sns.pairplot(pca_df, diag_kind='kde', hue='body_type', palette='tab10')

print('PCA on original')
plt.show()



In [ ]:
# apply standard scaling and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd


scaler = StandardScaler()
pca = PCA(n_components=32)

prod_with_support_list_scaled = scaler.fit_transform(prod_with_support_list.reshape(prod_with_support_list.shape[0], -1))
prod_with_support_list_pca = pca.fit_transform(prod_with_support_list_scaled)


fig, axs = plt.subplots(1, 1, figsize=(5, 5))

# plot explained variance
axs.plot(pca.explained_variance_ratio_, '.-')
axs.set_title('Explained variance ratio')


pca_df = pd.DataFrame(prod_with_support_list_pca[:, :3], columns=[f'PCA_{i}' for i in range(3)])
# pca_df['name'] = names_y
pca_df['body_type'] = [i // 10 for i in range(prod_with_support_list_pca.shape[0])]
# pca_df['pose'] = [i % 10 for i in range(C_xy_orig_list_pca.shape[0])]


# use numbers as markers
sns.pairplot(pca_df, diag_kind='kde', hue='body_type', palette='tab10')

print('PCA on product with support')
plt.show()



In [ ]:
supp_vec_list.shape

In [ ]:
# apply standard scaling and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd


scaler = StandardScaler()
pca = PCA(n_components=32)

# supp_vec_list_scaled = scaler.fit_transform(supp_vec_list.reshape(supp_vec_list.shape[0], -1))
# supp_vec_list_pca = pca.fit_transform(supp_vec_list_scaled)

supp_vec_list_scaled = scaler.fit_transform(
    (supp_vec_list.transpose(1, 2) @ supp_vec_list).reshape(supp_vec_list.shape[0], -1)
    )
supp_vec_list_pca = pca.fit_transform(supp_vec_list_scaled)


fig, axs = plt.subplots(1, 1, figsize=(5, 5))

# plot explained variance
axs.plot(pca.explained_variance_ratio_, '.-')
axs.set_title('Explained variance ratio')


pca_df = pd.DataFrame(supp_vec_list_pca[:, :3], columns=[f'PCA_{i}' for i in range(3)])
# pca_df['name'] = names_y
pca_df['body_type'] = [i // 10 for i in range(supp_vec_list_pca.shape[0])]
# pca_df['pose'] = [i % 10 for i in range(C_xy_orig_list_pca.shape[0])]


# use numbers as markers
sns.pairplot(pca_df, diag_kind='kde', hue='body_type', palette='tab10')

print('PCA on product with support')
plt.show()



In [ ]:
# supp_vec_list_product = supp_vec_list.transpose(1, 2) @ supp_vec_list
supp_vec_list_product = supp_vec_list_scaled.reshape(supp_vec_list.shape[0], feature_dim, feature_dim)

fig, axs = plt.subplots(1, 3, figsize=(15, 5))

axs_0 = axs[0].imshow(supp_vec_list_product[5], cmap='bwr')
plt.colorbar(axs_0, ax=axs[0])

axs_1 = axs[1].imshow(supp_vec_list_product[6], cmap='bwr')
plt.colorbar(axs_1, ax=axs[1])

axs_2 = axs[2].imshow(supp_vec_list_product[7], cmap='bwr')
plt.colorbar(axs_2, ax=axs[2])

plt.show()

In [ ]:
# apply standard scaling and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd


scaler = StandardScaler()
pca = PCA(n_components=32)

# supp_vec_list_scaled = scaler.fit_transform(supp_vec_list.reshape(supp_vec_list.shape[0], -1))
# supp_vec_list_pca = pca.fit_transform(supp_vec_list_scaled)

supp_vec_list_scaled = scaler.fit_transform(
    (supp_vec_list.transpose(1, 2) @ supp_vec_list).reshape(supp_vec_list.shape[0], -1)
    )
supp_vec_list_pca = pca.fit_transform(supp_vec_list_scaled)


fig, axs = plt.subplots(1, 1, figsize=(5, 5))

# plot explained variance
axs.plot(pca.explained_variance_ratio_, '.-')
axs.set_title('Explained variance ratio')


pca_df = pd.DataFrame(supp_vec_list_pca[:, :3], columns=[f'PCA_{i}' for i in range(3)])
# pca_df['name'] = names_y
pca_df['body_type'] = [i // 10 for i in range(supp_vec_list_pca.shape[0])]
# pca_df['pose'] = [i % 10 for i in range(C_xy_orig_list_pca.shape[0])]


# use numbers as markers
sns.pairplot(pca_df, diag_kind='kde', hue='body_type', palette='tab10')

print('PCA on product with support')
plt.show()

